In [1]:
from keras.models import Model
from keras.layers import Conv2D, Input, Conv2DTranspose, Activation, Convolution2D, Dropout
from keras.optimizers import adam
from keras.layers.merge import concatenate, add
#import prepare_data as pd
import numpy
import keras
from PIL  import Image

/home/wazir/.virtualenvs/wazir/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
scale = 4

def Residual_Block():
    Input_Shape = Input(shape=(None, None, 64))

    Convl1 = Conv2D(64,(3, 3), strides=(1, 1), padding='same', activation='relu')(Input_Shape)
    Convl1 = Conv2D(64,(3, 3), strides=(1, 1), padding='same', activation='linear')(Convl1)

    Out_Put = add(inputs=[Input_Shape, Convl1])
    Out_Put = Activation('relu')(Out_Put)

    model = Model(inputs=Input_Shape, outputs=Out_Put)

    return model

                          #BY DEFAULT SCALE 2
def model_WAZIR(scale):
    Input_Shape = Input(shape=(None,None, 1), name='input')

    Feature_Input = Convolution2D(64,(3, 3), strides=(1, 1), padding='same', activation='relu')(Input_Shape)
    Feature_Output = Residual_Block()(Feature_Input)

    #Upsampling U_S_1 = UpSampling1
    U_S_1 = Convolution2D(4,(1, 1), strides=(1, 1), padding='same', activation='relu')(Feature_Output)
    U_S_2 = Conv2DTranspose(4,(14, 14),strides=(scale,scale),padding='same', activation='relu')(U_S_1)
    U_S_3 = Convolution2D(64,(1, 1),strides=(1, 1), padding='same', activation='relu')(U_S_2)

    # RECONSTRUCTION OF MULTISCALE LAYERS
    R_C_L_1 = Residual_Block()(U_S_3)

    R_C_L_2 = Residual_Block()(R_C_L_1)

    #SUB RECONSTRUCTION LAYERS
    M_S_L_1 = Convolution2D(16,(1, 1), strides=(1, 1), padding='same', activation='relu',name='CONVOLUTION_MSL1')(R_C_L_2)

    M_S_L_2a = Convolution2D(16,(1, 1), strides=(1, 1),padding='same', activation='relu',name='CONVOLUTION_MSL2a')(M_S_L_1)

    M_S_L_2b = Convolution2D(16,(1, 3), strides=(1, 1),padding='same', activation='relu',name='CONVOLUTION_MSL2b')(M_S_L_1)
    M_S_L_2b = Convolution2D(16, (3, 1), strides=(1, 1),padding='same', activation='relu',name='CONVOLUTION_MSL2b1')(M_S_L_2b)

    M_S_L_2c = Convolution2D(16,(1, 5),strides=(1, 1),padding='same', activation='relu',name='CONVOLUTION_MSL2c')(M_S_L_1)
    M_S_L_2c = Convolution2D(16, (5, 1), strides=(1, 1),padding='same', activation='relu',name='CONVOLUTION_MSL2c1')(M_S_L_2c)

    M_S_L_2d = Convolution2D(16, (1, 7), strides=(1, 1),padding='same', activation='relu',name='CONVOLUTION_MSL2d')(M_S_L_1)
    M_S_L_2d = Convolution2D(16,(7, 1), strides=(1, 1), padding='same', activation='relu',name='CONVOLUTION_MSL2d1')(M_S_L_2d)


    M_S_L_2e = Convolution2D(16,(1, 7), strides=(1, 1),padding='same', activation='relu',name='CONVOLUTION_MSL2e')(M_S_L_1)
    M_S_L_2e = Convolution2D(16,(1,7), strides=(1, 1),padding='same', activation='relu',name='CONVOLUTION_MSL2e1')(M_S_L_2e)

    M_S_L_2f = Convolution2D(16,(1, 7), strides=(1, 1),padding='same', activation='relu',name='CONVOLUTION_MSL2f')(M_S_L_1)
    M_S_L_2f = Convolution2D(16,(7,1), strides=(1, 1),padding='same', activation='relu',name='CONVOLUTION_MSL2f1')(M_S_L_2f)

    M_S_L_2g = Convolution2D(16,(1, 7), strides=(1, 1),padding='same', activation='relu',name='CONVOLUTION_MSL2g')(M_S_L_1)
    M_S_L_2g = Convolution2D(16,(1,9), strides=(1, 1),padding='same', activation='relu',name='CONVOLUTION_MSL2g1')(M_S_L_2g)

   
    M_S_L_2 = concatenate(inputs=[M_S_L_2a, M_S_L_2b, M_S_L_2c, M_S_L_2d, M_S_L_2e,M_S_L_2f,M_S_L_2g])

    Out_Put = Convolution2D(1, (1, 1), strides=(1, 1), padding='same', activation='relu')(M_S_L_2)
    model = Model(input= Input_Shape, output=Out_Put)

    return model



def WAZIR_train(scale):
    WAZIR = model_WAZIR(scale) #IT IS DEPEND ON WHICH SCALE WE USE
    
    WAZIR.compile(optimizer=adam(lr=0.0003), loss='mse')
    #data, label = pd.read_training_data("./scale4.h5")
    #data, label = pd.read_training_data("./wazirdivx4.h5")
    #data, label = pd.read_training_data("./mixture.h5")
    #WAZIR.fit(data, label, batch_size=128, nb_epoch=20)############################CHANGE######################################################
    #WAZIR.save_weights("WAZIR_MODEL_SCALE_4C.h5")
    
def WAZIR_predict(scale):
    WAZIR = model_WAZIR(scale)                 #predict 4
    WAZIR.load_weights("wazir_training.h5")
    IMG_NAME = "Set5/baby_GT.bmp"
    INPUT_NAME = "INPUT_LOW_RESOLUTION_IMAGE.bmp"
    OUTPUT_NAME = "MODEL_OUTPUT_IMAGE.bmp"
    #OUTPUT_NAME = "INPUT_LOW_RESOLUTION_IMAGE.bmp"
    #INPUT_NAME = "MODEL_OUTPUT_IMAGE.bmp"

    import cv2 
    image = cv2.imread(IMG_NAME) #ORIGINAL GT IMAGE
    #image = cv2.resize(image, (255,255))#####################################################
    shape = image.shape
    image = cv2.resize(image, (shape[1] // scale, shape[0] // scale), cv2.INTER_CUBIC)
    cv2.imwrite(INPUT_NAME, image) #LOW RESOLUTION SCALED BY 4

    image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    Y = numpy.zeros((1, image.shape[0], image.shape[1], 1))
    Y[0, :, :, 0] = image[:, :, 0]
    image = cv2.resize(image, (shape[1], shape[0]), cv2.INTER_CUBIC)

    pre = WAZIR.predict(Y, batch_size=1)
    pre[pre[:] > 255] = 255
    pre = pre.astype(numpy.uint8)
    image[:, :, 0] = pre[0, :, :, 0]
    image = cv2.cvtColor(image, cv2.COLOR_YCrCb2BGR)
    cv2.imwrite(OUTPUT_NAME, image)

    # psnr calculation:
    img_1 = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
    img_1 = cv2.cvtColor(img_1, cv2.COLOR_BGR2YCrCb)
    img_2 = cv2.imread(OUTPUT_NAME, cv2.IMREAD_COLOR)
    img_2 = cv2.cvtColor(img_2, cv2.COLOR_BGR2YCrCb)

    print ("WAZIR_MODEL_PSNR")
    print (cv2.PSNR(img_1, img_2))



WAZIR_train(scale)
WAZIR_predict(scale)


Instructions for updating:
Colocations handled automatically by placer.


/home/wazir/.virtualenvs/wazir/lib/python3.5/site-packages/ipykernel_launcher.py:61: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=Tensor("in...)`


In [ ]:
LR_IMAGE = Image.open("INPUT_LOW_RESOLUTION_IMAGE.bmp")
MODEL_IMAGE = Image.open("./MODEL_OUTPUT_IMAGE.bmp")

In [ ]:
import cv2
img1 = cv2.imread('INPUT_LOW_RESOLUTION_IMAGE.bmp', cv2.IMREAD_UNCHANGED)
cv2.imshow('4X_LRIMAGE',img1)
cv2.waitKey(0) #image will not show until this is called
cv2.destroyAllWindows() #make sure window closes cleanly

img2 = cv2.imread('MODEL_OUTPUT_IMAGE.bmp', cv2.IMREAD_UNCHANGED)
cv2.imshow('4X_HRIMAGEimg',img2)
cv2.waitKey(0) #image will not show until this is called
cv2.destroyAllWindows() #make sure window closes cleanly